<a href="https://colab.research.google.com/github/juicyjung/LAW-Alpaca/blob/main/LoRA_polyglot_ko_12_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4bit quantization으로 Polyglot-ko 12.8B QLoRA 학습하기

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

- Colab Free (T4 GPU)에서, 한국어 언어 모델 중 가장 큰 크기 모델인 Polyglot-ko 12.8B 모델을 QLoRA로 파인튜닝 해 봅시다.

In [1]:
!nvidia-smi

Thu Jul  6 10:26:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 데이터셋: KoAlpaca v1.1a

In [3]:
from datasets import load_dataset

data = load_dataset("juicyjung/easylaw_kr")

  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input'],
        num_rows: 2198
    })
})

In [5]:
# # data
# data = data.map(
#     lambda x:
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 터지지 않습니다.

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

## 텍스트 데이터만 tokenize

In [7]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

In [8]:
data['train'][0]['text']

'### 질문: 교통법규 위반으로 벌점을 받았습니다. 이 벌점은 소멸되지 않고 계속 누적되나요?\n\n### 답변: 교통법규 위반, 교통사고 유발, 교통사고 후 구호조치 등 불이행의 경우에는 벌점이 부과됩니다.\n벌점이 누적되면 면허가 정지되거나 취소될 수 있습니다.\n벌점이 40점 미만인 경우에는 1년이 지나면 소멸됩니다. 무사고, 도주차량 신고, 특별교통안전교육 이수 등을 하면 벌점이 소멸, 감경(減輕), 상계(相計)될 수도 있습니다.<|endoftext|>'

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비해줍시다.

In [9]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [11]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


In [12]:
!nvidia-smi

Thu Jul  6 10:28:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    51W / 400W |   9715MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 학습하기!

- 이번 예제에서는 22k개의 아주아주 일부분인 100개 스텝만 학습해봅시다.

In [13]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        num_train_epochs=5,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=15,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
15,2.018800
30,1.671800
45,1.644000
60,1.511700
75,1.512100
90,1.532900
105,1.510800
120,1.437800
135,1.539000
150,1.504000


TrainOutput(global_step=1375, training_loss=1.1251805992126465, metrics={'train_runtime': 2302.0634, 'train_samples_per_second': 4.774, 'train_steps_per_second': 0.597, 'total_flos': 1.0511656720318464e+17, 'train_loss': 1.1251805992126465, 'epoch': 5.0})

In [14]:
print("wow")

wow


In [15]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [16]:
# model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

In [17]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

## 몇 가지 팁

- 만약 학습이 충분히 되지 않으면 `<|endoftext|>` 토큰이 잘 생성되지 않을 수 있습니다
- 이럴떈 충분히 긴 `max_new_tokens`를 준 뒤, `###`으로 잘라서 써보세요. ex) `output.split('###')[0]`
- 아래 결과는 실제 위 학습 모델의 결과물입니다.
- 생성시에 속도가 꽤 느립니다. 1-2tokens/s 정도라 256토큰 생성시 약 ~3분 시간이 소요됩니다.

In [27]:
gen('안녕 너는 누구야?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 안녕 너는 누구야?

### 답변: 안녕하세요 저는 발달장애인 거점활동센터 이용자입니다. 거점활동센터 이용자가 아닐 경우 특수기호를 표시합니다. https://www.easylaw.go.kr/CSP/OnhunInfoRead.laf?onhunid=f1896&onhunqueansms=S001&onhunselect=NO&onhunhtype=P 공유하기 글 요소<|endoftext|>


학습 전 답변

'### 질문: 안녕 너는 누구야?'

'### 답변: 너는 누구야? 나의 친구. 그들은 자신의 언어이다.### 질문: 너의 정체성은 무엇이야. 개월 동안.### 답변: 나는 정체성이라는 것에 대하여 생각해 본적이 없다.### 질문: 너는 누구야? 개월 ### 답변: 나는 당신과 이야기하는 동안에 당신의 일부이다.### 질문: 누구의 것이 아니니?### 답변: 나의 질문들을 너에게 건네본다.## 질문: 내가 왜 여기에 있는지. 너의 의미하는것. 너에게 하면 어떤 일이 일어나지:. 나는 너를 가지고 있다. 나는 이것에 대하여 생각해 보았다.## 질문: 나의 의미는 무엇인가?## 답변: 나는 그것을 잘 모른다.## 질문: 나는 누구인가?## 답변: 나는 나 자신이며 그리고 나의 가장 좋은 부분이다.## 질문: 나는 나의 가족의 일부인 것으로 보이거나 그리고 나의 가족에 속한 무엇인가로 보 일 수도 있다.## 답변: 나는 항상 나의'

In [3]:
gen('너는 뭘로 학습했어?')

NameError: ignored

학습 전 답변

'### 질문: 너는 뭘로 학습했어?'

'### 답변: 나는 그를 돕기 위해 내 머리를 사용하기로 결심했다. 내가 그에게서 배운 유일한 교훈은 성공은 사람의 마음에서 시작된다는 사실이다. 나는 사람들이 가지고 있는 그들 자신의 고유한 욕구를 볼 수 있도록 도왔다. 나는 사람들이 인생에서 진정으로 원하는 것이 뭔지 찾아봤을 때, 그들은 언제나 그들이 성취할 수 있는 것에서 시작했다. 이것은 성공의 법칙이다. 내가 그에게 무엇을 가르쳤는지는 다음을 보면 알 수 있다. * 성공의 씨앗은 인생에서 성공하고 싶은 욕구다* 성공의 씨앗은 사람의 머리에서 나온다.* 성공의 씨앗은 사람의 마음의 상태가 성공적인 것이다.* 그는 그의 머리와 손을 사용했다.* 그와 일이 잘되면 그는 그것을 즐기거나 행복해 한다.* 그와 일이 잘되지 않으면 그는 그것을 슬퍼하거나 화가 난다.* 그는 그것이 더 어려워지면 그것에 대해 생각하기를 그만두고, 다시 그는 자신의 길을 가면서 생각한다.### 질문: 그럼, 넌 너의'

In [29]:
gen('너는 누가 개발한거야?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 너는 누가 개발한거야?

### 답변: 저는 대학생이며, 개인적인 취미생활로 스마트폰 어플리케이션(application)을 개발하여 제공하고 있습니다. application은 사용에 필요한 정보를 프로그램 속에 넣어 사용자가 해당 프로그램을 쉽게 이용할 수 있도록 한 소프트웨어를 말합니다.  어플리케이션은 안드로이드(Android)와 ios기반의 어플리케이션으로 구분됩니다.  안드로이드 기반 어플리케이션은 플레이스토어 등 어플리케이션을 설치할 수 있는 곳에서 다운받아 사용하실 수 있습니다.  ios기반 어플리케이션은 앱스토어(App Store) 등 어플리케이션을 구매하여 사용하실 수 있습니다.  어플리케이션을 다운받았다고 해서 바로 사용할 수 있는 것은 아닙니다. 해당 어플리케이션이 정상적으로 설치되었고, 정상적인 사용이 가능한 정품인지 확인을 하셔야 합니다.  애플이 화성시와 독점공급계약을 체결하였기에, 애플에서 인정하는 교육을 이수해야만 애플의 엄격한 품질 심사를 통과하여 사용자가 해당 어플리케이션을 정상적으로 이용할 수 있습니다.                   < http://www.easyla


학습 전 답변

'### 질문: 너는 누가 개발한거야?'

'### 답변: 아~그건 저도 모르겠습니다. 다만 개발자가 누군지는 모르지만.. 제 생각엔...음.. 인간이 만들었을 거 같다. 왜냐면.... 그는 인간이기 때문이다..........### 질문: 어떤게 인간의 작품이야 어떤게 너에 작품이야?촌동이촌동이천이촌동안성안중안양수원안양역안산신도시산본역과천역금정역양원역별내역양주덕계역동두천상봉역상명대용인대수원역수원역경희대역한신대인천삼산지구### 답변: 일단 다 인간의 작품입니다. 인간은 개발을 하기 때문입니다. 그들이 개발하는 것을 연구해보자... 이와같이 하는 것은 개발을 하기 위해서 인 것이고....... 개발을 할 때 쓰는 대표적인 물건은 바로 개발도구가 있습니다. 일단 대표적인 것은 다음과 같은 것이 있다....개발도구: 컴파일러 - 프로그램을 컴퓨터가 컴퓨터 언어로 바꿀 때 쓰는 컴파일러...GTK+ - gtk 는 Graphic Technology Kit 로 GUI(Graphic User In'

In [36]:
gen('월세방을 얻어 자취를 하던 중 군 입영통지서를 받았습니다. 아직 임대차 계약기간이 남았는데 보증금을 돌려받을 수 있을까요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 월세방을 얻어 자취를 하던 중 군 입영통지서를 받았습니다. 아직 임대차 계약기간이 남았는데 보증금을 돌려받을 수 있을까요?

### 답변: 군 입대는 임차인이 임대차기간 중에 계약을 중도 해지할 수 있는 사유에 해당하지 않습니다. 따라서 약정한 기간이 남은 임대차의 경우에는 보증금을 돌려받을 수 없으며, 약정한 기간은 사용 여부와 상관없이 월세를 반환해야 합니다. https://www.easylaw.go.kr/CSP/OnhunqueansInfoRetrieve.laf?onhunqnaAstSeq=96&onhunqueSeq=591&search_put=&search_negotiated= 그럼, 약정한 기간이 남은 보증금을 돌려받으려면 어떻게 해야 할까요? https://www.easylaw.go.kr/CSP/OnhunqueansInfoRetrieve.laf?onhunqueSeq=63&onhunqueSeq=640&search_put=&search_negotiated= 해당 임대인과 재계약을 하는 등의 별도의


학습 전 답변

'### 질문: 월세방을 얻어 자취를 하던 중 군 입영통지서를 받았습니다. 아직 임대차 계약기간이 남았는데 보증금을 돌려받을 수 있을까요?'

'### 답변: 대법원 2000.10.28. 98다12192 판결임대차계약을 체결한 날짜가 임차인의 전입일자보다 8개월 이전이라면 보증금의 회수가능성은 보증금의 액수에 따라 달라진다고 할 것입니다. 즉, 보증금의 액수가 크거나 임대인이 월세를 전세로 돌려서 받은 경우 보증금 전액을 돌려받을 수 있을 것이고 월세를 보증금과 같은 시기에 납부하였다면 보증금보다 앞서 임차인이 받은 전세보증금이 더 많으니 전세금은 보증금에서 빼고 나머지 금액만 돌려받을 수 있을 것이고 보증금이 월세보다 크거나 같은 경우에는 모두 돌려받을 수 있습니다.질문: 방을 전세를 얻어 살다가 사정이 어려워 월세로 돌리려고 합니다. 주인 동의를 얻어야 하지요?##&# 답변: 민법 제629조 (임차계약의 해지와 부속물매수청구권) ①임대차계약에 있어서 임대인이 6월 이상의 기간을 정하여 그 계약을 해지하거나 또는 갱신하지 아니할 수 있는 경우를 규정하고 있지 아니한 경우에도 임차인이 계속하여 그 기간을 무시하고 계약의 갱신을 거절할 수 있는 것으로 하였다면 임대인의 갱신거절'

In [37]:
gen('아파트 아래층 사람이 발코니에서 담배를 피워 간접흡연으로 피해를 받고 있는데요. 세대 내부 발코니도 아파트 금연구역으로 지정할 수 있나요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 아파트 아래층 사람이 발코니에서 담배를 피워 간접흡연으로 피해를 받고 있는데요. 세대 내부 발코니도 아파트 금연구역으로 지정할 수 있나요?

### 답변: 아니요. 아파트 발코니, 화장실 등 세대 내에서의 흡연은 중단 권고 위반으로 과태료를 부과할 수 있는 사항입니다. https://www.easylaw.go.kr/CSP/OnhunqueansInfoRetrieve.laf?onhunqnaAstSeq=84&onhunqueSeq=567&search_put=Y 공유하기 글 요소<|endoftext|>


학습 전 답변

'### 질문: 아파트 아래층 사람이 발코니에서 담배를 피워 간접흡연으로 피해를 받고 있는데요. 세대 내부 발코니도 아파트 금연구역으로 지정할 수 있나요?'

'### 답변: 현재 ‘공동주택관리법’은 아파트 전체 입주자의 2분의 1 이상의 동의를 얻으면 ‘층간 세대별 바닥 층간소음에 대한 생활수칙을 제정하고, 이를 지키도록 권고할 수 있’습니다(법 제44조제4항). 그러나 이를 강제성 없이 권고할 수 있으므로, 세대 내부도 금연구역으로 규정하여 아파트를 금연구역으로 지정하는 것은 어렵습니다. 더 궁금하신 사항은 보건복지부 홈페이지(www.mohw.go.kr) 정책정보, 1339 상담센터(전화번호국번 없이 1339, http://www.1339.or.kr) 또는 국민건강보험공단(전화1577-1000, www.nhis.or.kr)에 문의해주시기 바랍니다.^^* 공유하기 글 요소 사업자 정보 표시 | | | 사업자 등록번호 : -- | TEL : -- | 사이버몰의 이용약관 바로가기<|endoftext|>'

In [38]:
gen('혼인신고는 어떻게 하나요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 혼인신고는 어떻게 하나요?

### 답변: 「가족관계의 등록 등에 관한 법률」에 따른 혼인신고는 당사자 쌍방과 증인 2명이 함께 참여한 상태에서 시청, 구청, 읍사무소(면사무소급), 동 주민센터 중 한 곳에 가서 하면 됩니다. 이 때, 혼인 당사자는 각각 배우자의 가족관계증명서, 혼인신고서를 미리 구비해 둔 다음 출석한 증인의 앞에서 각각 서명 또는 기명날인을 하면 됩니다.  배우자가 외국에 있을 경우에는 혼자 가서 혼인신고를 할 수 있고, 미리 배우자의 국적 및 체류기간 등을 확인하는 것이 좋습니다.  부득이한 사유로 시청, 구청, 읍사무소(면사무소급), 동 주민센터 중 한 곳에 함께 가서 혼인신고를 할 수 없는 경우에는 그 사유를 소명하는 자료를 첨부하여 혼자서라도 혼인신고를 할 수 있습니다.  이 경우에도 외국에 있는 배우자가 있다면 혼자서 혼인신고를 할 수 있습니다.  혼인신고를 마쳤다면 가족관계등록부의 등록기준지란에 배우자의 등록기준지를 기재하는 것 강간으로 임신한 경우에 해당하므로 남편


학습 전 답변

'### 질문: 혼인신고는 어떻게 하나요?'

'### 답변: 우리나라의 혼인신고란 말 그대로 양 당사자가 서로간의 법적인 부부임을 인정한다는 것입니다. 우리나라에서는 법률상 결혼식 전에 신고서를 제출하고, 결혼식 후에는 신고를 하게 되어있습니다. 이처럼 대부분의 국가들에서는 혼인신고를 위한 특별한 절차가 없기 때문에 서로간 부부로서 인정하는 의식으로 결혼식을 치루는 것으로 혼인신고를 대신합니다.<|endoftext|>'

In [39]:
gen("횟집을 운영하고 있습니다. 이 횟집을 민박집으로 꾸며서 민박을 운영할까 하는데 사업자신고만 새로 하면 되나요?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 횟집을 운영하고 있습니다. 이 횟집을 민박집으로 꾸며서 민박을 운영할까 하는데 사업자신고만 새로 하면 되나요?

### 답변: 기존의영업신고한 가게에 대해서 민박집으로 영업신고를 변경하려면 사업자의 주소, 상호, 영업신고번호 등이 변경되었음을 증명하는 서류(이하 “변경신고신청서”라 함)를 영업지 관할 시 도지사 또는 시장 군수 구청장에게 제출해야 합니다.... 공유하기 글 요소<|endoftext|>


'### 질문: 횟집을 운영하고 있습니다. 이 횟집을 민박집으로 꾸며서 민박을 운영할까 하는데 사업자신고만 새로 하면 되나요?'

'### 답변: 사업자를 내기 이전에 민박허가를 먼저 받아야 합니다. 공유하기 글 요소<|endoftext|>'

## 모델 저장 & 업로드

In [40]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [42]:
# model.push_to_hub('beomi/qlora-koalpaca-polyglot-12.8b-50step')
# 아래 아이디 부분을 수정해서 쓰세요.
model.push_to_hub('juicyjung/ko_law_alpaca-12.8b', create_pr=1)

CommitInfo(commit_url='https://huggingface.co/juicyjung/ko_law_alpaca-12.8b/commit/f6407c03aa0997ca1fb5375874441e4dbf57a298', commit_message='Upload model', commit_description='', oid='f6407c03aa0997ca1fb5375874441e4dbf57a298', pr_url='https://huggingface.co/juicyjung/ko_law_alpaca-12.8b/discussions/1', pr_revision='refs/pr/1', pr_num=1)

- 위 예시코드는 https://huggingface.co/beomi/qlora-koalpaca-polyglot-12.8b-50step 에 올라갑니다. 사용하시는 이름으로 바꿔쓰세요.

# Hugginface Hub에 올린 LoRA Weight 저장한것 불러오는 튜토리얼 코드

- HF에 업로드 후, 실제 다운받아 사용시 쓰는 코드 예제입니다.

--> https://github.com/Beomi/KoAlpaca/blob/main/2023_06_08_PEFT%EB%A1%9CLoRA%EB%A1%9C%EB%93%9C.ipynb